<a href="https://colab.research.google.com/github/ashimasinghal/Indian-NER/blob/main/bengali_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seqeval
!pip install transformers

     |████████████████████████████████| 43 kB 2.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=8283fe6dc4c8803e3083d0be097b0e0ff243523488f37ef31a110181296ca6fb
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
     |████████████████████████████████| 3.1 MB 36.1 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [ ]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [ ]:
!pip list | grep -E 'transformers|torch|Keras'

Keras-Preprocessing           1.1.2
torch                         1.10.0+cu111
torchaudio                    0.10.0+cu111
torchsummary                  1.5.1
torchtext                     0.11.0
torchvision                   0.11.1+cu111
transformers                  4.12.5


In [ ]:
# load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BERT/NER Corpus/Bengali.csv')

Mounted at /content/drive


In [ ]:
df

,Unnamed: 0,word,T1,T2,T3
0,0,রহমান,B-PERSON,B-INDIVIDUAL,B-FAMILYNAME
1,1,ভাষা,o,o,o
2,2,দফা,o,o,o
3,3,স্বাধীনতা,o,o,o
4,4,যুদ্ধ,o,o,o
...,...,...,...,...,...
49519,49519,পূজামণ্ডপে,o,o,o
49520,49520,বিজযা,o,o,o
49521,49521,দশমীতে,o,o,o
49522,49522,দেবীবরণের,o,o,o


In [ ]:
sentence_id = []
words = df['word']
sentence = 1
for word in words:
  sentence_id.append(sentence)
  if(word=='.'):
    sentence += 1

In [ ]:
df.insert(1,'Sentence ID',sentence_id,allow_duplicates=True)
df.head(50)

,Unnamed: 0,Sentence ID,word,T1,T2,T3
0,0,1,রহমান,B-PERSON,B-INDIVIDUAL,B-FAMILYNAME
1,1,1,ভাষা,o,o,o
2,2,1,দফা,o,o,o
3,3,1,স্বাধীনতা,o,o,o
4,4,1,যুদ্ধ,o,o,o
5,5,1,.,o,o,o
6,6,2,বাংলাদেশের,B-LOCATION,B-PLACE,B-NATION
7,7,2,স্বাধীনতা,o,o,o
8,8,2,আন্দোলন,o,o,o
9,9,2,],o,o,o


In [ ]:
hindi = pd.DataFrame()
hindi = df[['Sentence ID','word','T3','T1']]
hindi.rename(columns={'Sentence ID': 'Sentence', 'T3': 'POS','T1':'TAG'}, inplace=True)
hindi

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Sentence,word,POS,TAG
0,1,রহমান,B-FAMILYNAME,B-PERSON
1,1,ভাষা,o,o
2,1,দফা,o,o
3,1,স্বাধীনতা,o,o
4,1,যুদ্ধ,o,o
...,...,...,...,...
49519,339,পূজামণ্ডপে,o,o
49520,339,বিজযা,o,o
49521,339,দশমীতে,o,o
49522,339,দেবীবরণের,o,o


In [ ]:
hindi.POS.unique()

array(['B-FAMILYNAME', 'o', 'B-NATION', 'B-TITLE', 'I-NATION', 'B-CITY',
       'B-MUSEUM', 'I-MUSEUM', 'B-DISTRICT', 'B-PARAMILITARY', 'I-TITLE',
       'B-CEREAL', 'B-VEG', 'B-FRUIT', 'I-DISTRICT', 'I-FAMILYNAME',
       'I-CITY', 'B-STATE', 'B-', 'B-PARK', 'I-PARK', 'I-', 'B-RELPLACE',
       'I-RELPLACE', 'B-CONTINENT', 'B-MONUMENT', 'I-MONUMENT',
       'I-CONTINENT', 'B-CONTINENTAL', 'I-CONTINENTAL', 'B-ASSOCIATION',
       'I-ASSOCIATION'], dtype=object)

In [ ]:
len(hindi.TAG.unique())

51

In [ ]:
hindi['Sentence'].nunique(), hindi.word.nunique(), hindi.POS.nunique(), hindi.TAG.nunique()

(339, 14512, 32, 51)

In [ ]:

hindi.TAG.value_counts()

o                  35421
B-LOCATION          4853
I-LOCATION          2444
B-PERSON            1063
I-FACILITIES         725
B-DISTANCE           607
I-DISTANCE           595
B-FACILITIES         430
I-PERSON             386
I-ORGANIZATION       259
B-ORGANIZATION       218
B-YEAR               197
B-COUNT              196
I-LIVTHINGS          195
B-LIVTHINGS          187
I-YEAR               168
B-ENTERTAINMENT      166
I-PERIOD             141
B-PERIOD             139
I-ENTERTAINMENT      126
B-MONTH              116
B-PLANTS             107
B-ARTIFACT           104
I-ARTIFACT            93
B-MATERIALS           79
B-TIME                74
I-PLANTS              62
I-COUNT               60
I-MATERIALS           45
I-TIME                44
I-DATE                34
B-LOCOMOTIVE          30
B-DATE                29
B-DAY                 27
B-QUANTITY            17
I-QUANTITY            16
B-SDAY                13
I-MONTH               10
I-NUMEX                8
I-LOCOMOTIVE           7


In [ ]:
# parsing the data


In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# hindi_train, hindi_test = train_test_split( hindi, test_size=0.33, random_state=42)

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["TAG"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(hindi)

In [ ]:
sentences = [[s[0] for s in sent] for sent in getter.sentences]
sentences[0]

['রহমান', 'ভাষা', 'দফা', 'স্বাধীনতা', 'যুদ্ধ', '.']

In [ ]:
poses = [[s[1] for s in sent] for sent in getter.sentences]
print(poses[0])

['B-FAMILYNAME', 'o', 'o', 'o', 'o', 'o']


In [ ]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['B-PERSON', 'o', 'o', 'o', 'o', 'o']


In [ ]:
# Make TAG name into index for training
tags_vals = list(set(hindi["TAG"].values))
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')
tags_vals = set(tags_vals)
tags_vals

{'B-ARTIFACT',
 'B-COUNT',
 'B-DATE',
 'B-DAY',
 'B-DISEASE',
 'B-DISTANCE',
 'B-ENAMEX',
 'B-ENTERTAINMENT',
 'B-FACILITIES',
 'B-LIVTHINGS',
 'B-LOCATION',
 'B-LOCOMOTIVE',
 'B-MATERIALS',
 'B-MONEY',
 'B-MONTH',
 'B-NUMEX',
 'B-ORGANIZATION',
 'B-PERIOD',
 'B-PERSON',
 'B-PLANTS',
 'B-QUANTITY',
 'B-SDAY',
 'B-TIME',
 'B-TIMEX',
 'B-YEAR',
 'B-null',
 'I-ARTIFACT',
 'I-COUNT',
 'I-DATE',
 'I-DAY',
 'I-DISTANCE',
 'I-ENAMEX',
 'I-ENTERTAINMENT',
 'I-FACILITIES',
 'I-LIVTHINGS',
 'I-LOCATION',
 'I-LOCOMOTIVE',
 'I-MATERIALS',
 'I-MONEY',
 'I-MONTH',
 'I-NUMEX',
 'I-ORGANIZATION',
 'I-PERIOD',
 'I-PERSON',
 'I-PLANTS',
 'I-QUANTITY',
 'I-SDAY',
 'I-TIME',
 'I-TIMEX',
 'I-YEAR',
 'X',
 '[CLS]',
 '[SEP]',
 'o'}

In [ ]:
tag2idx={
'B-ARTIFACT':1,
 'B-COUNT':2,
 'B-DATE':3,
 'B-DAY':4,
 'B-DISEASE':5,
 'B-DISTANCE':6,
 'B-ENAMEX':7,
 'B-ENTERTAINMENT':8,
 'B-FACILITIES':9,
 'B-LIVTHINGS':10,
 'B-LOCATION':11,
 'B-LOCOMOTIVE':12,
 'B-MATERIALS':13,
 'B-MONEY':14,
 'B-MONTH':15,
 'B-NUMEX':16,
 'B-ORGANIZATION':17,
 'B-PERIOD':18,
 'B-PERSON':19,
 'B-PLANTS':20,
 'B-QUANTITY':21,
 'B-SDAY':22,
 'B-TIME':23,
 'B-TIMEX':24,
 'B-YEAR':25,
 'B-null':27,
 'I-ARTIFACT':26,
 'I-COUNT':28,
 'I-DATE':29,
 'I-DAY':30,
 'I-DISTANCE':31,
 'I-ENAMEX':32,
 'I-ENTERTAINMENT':33,
 'I-FACILITIES':34,
 'I-LIVTHINGS':35,
 'I-LOCATION':36,
 'I-LOCOMOTIVE':37,
 'I-MATERIALS':38,
 'I-MONEY':39,
 'I-MONTH':40,
 'I-NUMEX':41,
 'I-ORGANIZATION':42,
 'I-PERIOD':43,
 'I-PERSON':44,
 'I-PLANTS':45,
 'I-QUANTITY':46,
 'I-SDAY':47,
 'I-TIME':48,
 'I-TIMEX':49,
 'I-YEAR':50,
 'X':51,
 '[CLS]':52,
 '[SEP]':53,
 'o':54
 }
tag2idx

{'B-ARTIFACT': 1,
 'B-COUNT': 2,
 'B-DATE': 3,
 'B-DAY': 4,
 'B-DISEASE': 5,
 'B-DISTANCE': 6,
 'B-ENAMEX': 7,
 'B-ENTERTAINMENT': 8,
 'B-FACILITIES': 9,
 'B-LIVTHINGS': 10,
 'B-LOCATION': 11,
 'B-LOCOMOTIVE': 12,
 'B-MATERIALS': 13,
 'B-MONEY': 14,
 'B-MONTH': 15,
 'B-NUMEX': 16,
 'B-ORGANIZATION': 17,
 'B-PERIOD': 18,
 'B-PERSON': 19,
 'B-PLANTS': 20,
 'B-QUANTITY': 21,
 'B-SDAY': 22,
 'B-TIME': 23,
 'B-TIMEX': 24,
 'B-YEAR': 25,
 'B-null': 27,
 'I-ARTIFACT': 26,
 'I-COUNT': 28,
 'I-DATE': 29,
 'I-DAY': 30,
 'I-DISTANCE': 31,
 'I-ENAMEX': 32,
 'I-ENTERTAINMENT': 33,
 'I-FACILITIES': 34,
 'I-LIVTHINGS': 35,
 'I-LOCATION': 36,
 'I-LOCOMOTIVE': 37,
 'I-MATERIALS': 38,
 'I-MONEY': 39,
 'I-MONTH': 40,
 'I-NUMEX': 41,
 'I-ORGANIZATION': 42,
 'I-PERIOD': 43,
 'I-PERSON': 44,
 'I-PLANTS': 45,
 'I-QUANTITY': 46,
 'I-SDAY': 47,
 'I-TIME': 48,
 'I-TIMEX': 49,
 'I-YEAR': 50,
 'X': 51,
 '[CLS]': 52,
 '[SEP]': 53,
 'o': 54}

In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
# training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

0

In [ ]:
# vocabulary = "/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased/bert-base-multilingual-cased-vocab.txt"

In [ ]:
max_len  = 45
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
# tokenizer = AutoTokenizer.from_pretrained("skillzzzzzy/bengberto")

In [ ]:
# max_len  = 45
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased', truncation=True, do_lower_case=True)
# # tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
# # model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

In [ ]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:16
texts:[CLS] র ##হ ##মান ভাষা দ ##ফা সব ##া ##ধীন ##তা য ##দ ##ধ . [SEP]
No.0,len:16
lables:[CLS] B-PERSON X X o o X o X X X o X X o [SEP]
No.1,len:2568
texts:[CLS] বাংলাদেশের সব ##া ##ধীন ##তা আ ##ন ##দে ##াল ##ন ] ] স ##নের ছ ##াত ##র ##নে ##তা , শ ##ে ##খ ম ##জি ##ব ] ] সালে শ ##ে ##খ ম ##জি ##ব ] ] সালে হ ##সে ##ন শ ##হী ##দ সে ##াহ ##রা ##ও ##য ##ার ##দ ##ীর সাথে শ ##ে ##খ ম ##জি ##ব ] ] সালে হ ##সে ##ন শ ##হী ##দ সে ##াহ ##রা ##ও ##য ##ার ##দ ##ীর সাথে শ ##ে ##খ ম ##জি ##ব ] ] অ ##ন ##য ##ান ##য র ##াজ ##নৈতিক দল ##গ ##লে ##ার সাথে কাজ করতে গ ##ি ##য ##ে , ম ##জি ##ব আ ##ই ##য ##ব ব ##িরে ##া ##ধ ##ী দল পর ##ার ##থ ##ী ফ ##াত ##ি ##মা জ ##িন ##না ##হ ##কে স ##ম ##র ##থ ##ন করেন য ##থা ##রী ##তি ন ##ির ##বা ##চ ##নের দ ##ই স ##প ##তা ##হ পর ##বে তাকে আ ##ট ##ক করা হ ##য তাকে র ##া ##ষ ##ট ##র ##দ ##রে ##াহ ##ীত ##া এবং আ ##প ##ত ##তি ##ক ##র পর ##স ##তা ##ব প ##েশ ##ের অ ##ভি ##য ##ে ##াগ ##ে অ ##ভি ##য ##ক ##ত করে এক বছরের ক ##ার ##দ ##ণ ##ড ##ে দ ##ণ ##ড ##িত করা অব #

In [ ]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[   101    974  26145  29454  72461    965  84929 100024  12079  90267
  19668    973  17511  56251    119    102      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0]


In [ ]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["o"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])


[52 19 51 51 54 54 51 54 51 51 51 54 51 51 54 53 54 54 54 54 54 54 54 54
 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(237, 102, 237, 102)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [ ]:
batch_num = 32

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [ ]:
# model = AutoModelForMaskedLM.from_pretrained("skillzzzzzy/bengberto" ,num_labels=len(tag2idx))

In [ ]:
# # model_file_address = '/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased'
model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased",num_labels=len(tag2idx))

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
m_labels = len(tag2idx)

In [ ]:
model;

In [ ]:
# model.cuda();

In [ ]:
epochs = 30
max_grad_norm = 1.0

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# fine tuning
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
model.train();

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 237
  Batch size = 32
  Num steps = 240


Epoch:   0%|          | 0/30 [00:04<?, ?it/s]


IndexError: ignored

In [ ]:
bert_out_address = 'ashiNLP/bert_bengberto_model/bengali'

In [ ]:
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model

In [ ]:
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [ ]:
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('ashiNLP/bert_bengberto_model/bengali/vocab.json',
 'ashiNLP/bert_bengberto_model/bengali/merges.txt')

In [ ]:
# load model
model = AutoModelForMaskedLM.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [ ]:
# model.cuda();

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# EVALUATION

In [ ]:
model.eval();

In [ ]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
# output_eval_file = os.path.join('/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased', "eval_results.txt")
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =102
  Batch size = 32


In [ ]:
save_model_address = 'ashiNLP/bert_bengberto_model/bengali'

In [ ]:
save_model = AutoModelForMaskedLM.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_model_address,do_lower_case=False)

In [ ]:
# predict
test_query = "আমি 20 বছর ধরে দিল্লিতে থাকি"

In [ ]:
tokenized_texts = []

temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)

token_list

In [ ]:
for m,token in enumerate(token_list):
    temp_token.append(token)

In [ ]:
if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]

In [ ]:
temp_token.append('[SEP]')
temp_token

In [ ]:
tokenized_texts.append(temp_token)

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:

segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
segment_ids = torch.tensor(segment_ids)

In [ ]:
model.eval();

In [ ]:
with torch.no_grad():
        outputs = save_model(input_ids, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [ ]:
predict_results = logits.detach().cpu().numpy()
predict_results.shape


In [ ]:
from scipy.special import softmax
result_arrays_soft = softmax(predict_results[0])
result_arrays_soft[0]


In [ ]:
result_array = result_arrays_soft
len(result_array),len(result_array[0])

In [ ]:
# Get each token final predict tag index result
result_list = np.argmax(result_array,axis=-1)
result_list

In [ ]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()